In [ ]:
import numpy as np 
import pandas as pd
import ast

In [ ]:
credits_df = pd.read_csv("credits.csv")
movies_df = pd.read_csv("movies.csv")
print(credits_df.shape)
print(movies_df.shape)

In [ ]:
#pd.set_option('display.max_columns',None)
#pd.set_option('display.max_rows',None)


In [ ]:
credits_df.head()

In [ ]:
movies_df = movies_df.merge(credits_df,on='title')
movies_df.head()

In [ ]:
movies_df.info()

In [ ]:
movies_df = movies_df[['movie_id','title','overview','genres','keywords','cast','crew']]
movies_df

In [ ]:
movies_df.info()

In [ ]:
movies_df.isnull().sum()#check how many null value there are

In [ ]:
movies_df.dropna(inplace=True) # delete null value
movies_df.isnull().sum()# there is no null value


In [ ]:
movies_df.duplicated().sum()
movies_df.drop_duplicates()
movies_df.duplicated().sum() # give the ,umber of dublicated data

In [ ]:
def convert(obj):
    l = []
    for i in ast.literal_eval(obj):
        l.append(i['name'])
    return l

In [ ]:
movies_df['genres'] = movies_df['genres'].apply(convert)
movies_df['keywords'] = movies_df['keywords'].apply(convert)
movies_df.head()

In [ ]:
def convert3(obj):
    l = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter !=3 :
            l.append(i['name'])
            counter +=1
        else:
            break
    return l

In [ ]:
movies_df['cast'] = movies_df['cast'].apply(convert3)


In [ ]:
movies_df.head()

In [ ]:
def fetch_director(obj):
    l=[]
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            l.append(i['name'])
    return l

In [ ]:
movies_df['crew'] = movies_df['crew'].apply(fetch_director)
movies_df.head()

In [ ]:
movies_df['overview'] = movies_df['overview'].apply(lambda x : x.split())

In [ ]:
movies_df['genres'] = movies_df['genres'].apply(lambda x: [i.replace(" ","") for i in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x: [i.replace(" ","") for i in x])
movies_df['cast'] = movies_df['cast'].apply(lambda x: [i.replace(" ","") for i in x])
movies_df['crew'] = movies_df['crew'].apply(lambda x: [i.replace(" ","") for i in x])

In [ ]:
movies_df['tags']= movies_df['overview']+movies_df['genres']+movies_df['keywords']+movies_df['cast']+movies_df['crew']

In [ ]:
new_def = movies_df[['movie_id','title','tags']]

In [ ]:
new_def['tags'] = new_def['tags'].apply(lambda x:' '.join(x))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000 , stop_words='english') 

In [ ]:
cv.fit_transform(new_def['tags']).toarray().shape

In [ ]:
vectors = cv.fit_transform(new_def['tags']).toarray()


In [ ]:
vectors[0]

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
new_def['tags'] = new_def['tags'].apply(stem)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
cosine_similarity(vectors)

In [44]:
cosine_similarity(vectors).shape

(4805, 4805)

In [45]:
similarity = cosine_similarity(vectors)

In [46]:
similarity[0]

array([1.        , 0.08964215, 0.05976143, ..., 0.02548236, 0.02672612,
       0.        ])

In [48]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:6]

[(539, 0.26089696604360174),
 (507, 0.25302403842552984),
 (1192, 0.2522624895547565),
 (1214, 0.2480694691784169),
 (582, 0.24397501823713333)]

In [55]:
def recommend(movie) : 
    movie_index = new_def[new_def['title']==movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    for i in movies_list :
        print(new_def.iloc[i[0]].title)

In [53]:
recommend('Avatar')

Titan A.E.
Independence Day
Small Soldiers
Aliens vs Predator: Requiem
Battle: Los Angeles


In [57]:
recommend('Captain America: Civil War')

Captain America: The First Avenger
Captain America: The Winter Soldier
Iron Man 3
Avengers: Age of Ultron
Iron Man 2


In [58]:
recommend('What Planet Are You From?')

Escape from Planet Earth
Planet 51
Alien
Meet Dave
The Day the Earth Stood Still


In [59]:
recommend('Alien')

The Texas Chainsaw Massacre 2
Darkness
Hayride
Ri¢hie Ri¢h
The Hills Have Eyes
